In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
import asyncio

In [3]:
def msg_add(msg1 : list,msg2):
    print(msg1,msg2,sep=", \n")
    temp = msg2+msg1
    print(temp)
    return temp

class State(TypedDict): # create a type bound dictionary type 

    messages : Annotated[list , add_messages] # create a key in the dict type with additional metadata added using Annotated
    # This basically means that it instructs langgraph to update message using the add_message function
    # function like add_message are called reducers

graph = StateGraph(State)

Now our chatbot can do two key things:
- Each node can take the state variable as input and output an update to the state variable
- Every time a node updates the message list of the state variable the messages are appended instead of just overwriting over the previous messages.

In [4]:
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
load_dotenv()

model  = init_chat_model(model="deepseek/deepseek-chat-v3-0324:free" , model_provider="openai")

# creating the node function
def llm(state: State):
    return {"messages" : model.invoke(state["messages"])}

# adding the node 
# the first argument is the unique node name 
# second argument is the callback function whenever the node is called
graph.add_node("llm" , llm)

In [5]:
graph.add_edge(START , "llm") # tell the graph where to start 
graph.add_edge("llm" ,END) # tell the graph where to end i.e. anytime this node is run you can exit

In [6]:
compiledGraph = graph.compile() # compile the whole graph to use invoke in the script

#### Now building the chatbot

In [ ]:
while True:
    user_inp = input("User: ")
    if not user_inp == "q":
        output = compiledGraph.invoke({
            "messages" : 
                {
                    "role": "user",
                    "content": user_inp
                },
            }
        )["messages"][-1].content
        print( "Assistant: ", output, flush=True)
    else:
        break
        

Hi there! 😊 How can I help you today?
Hey there! 😊 Right now, I'm analyzing your message and generating a response—thinking about how best to assist you! I don't have thoughts or feelings, but my "goal" is to provide helpful, friendly, and accurate answers to whatever you need.  

What about you—what brings you here today? Need help with something? Or just curious? Ask away! 🚀
